docker run -d --name weaviate -p 28080:8080 -e ENABLE_MODULES=text2vec-transformers -e TRANSFORMERS_INFERENCE_API=http://host.docker.internal:7000/encode semitechnologies/weaviate:latest


In [ ]:
!pip install weaviate-client


In [ ]:
import weaviate

client = weaviate.Client("http://localhost:28080")
client.schema.delete_class("Sale")
schema = {
    "classes": [
        {
            "class": "Sale",
            "vectorizer": "text2vec-transformers",
            "properties": [
                {
                    "name": "product_name",
                    "dataType": ["text"],
                    "description": "Name of the product",
                },
                {
                    "name": "company",
                    "dataType": ["text"],
                    "description": "Name of the company",
                },
                {
                    "name": "year",
                    "dataType": ["int"],
                    "description": "Year of production",
                },
                {
                    "name": "price",
                    "dataType": ["number"],
                    "description": "Price of the product",
                },
            ],
        }
    ]
}

client.schema.delete_all()

client.schema.create(schema)

In [ ]:
# Get the current schema
current_schema = client.schema.get()

# Print the entire schema
print(current_schema)


In [ ]:
# Sample data for the "Sales" class
sales_data = [
    {"year": 2024, "company": "lnt", "product_name": "watch 10", "amount": 55000},
    {"year": 2034, "company": "lnt", "product_name": "iwatch 9", "amount": 15000},
    {
        "year": 2054,
        "company": "capgemini",
        "product_name": "iphone|iwatch|apple watch",
        "amount": 5000,
    },
    {"year": 2014, "company": "tcs", "product_name": "red mi - T5", "amount": 52000},
    {
        "year": 2014,
        "company": "tcs",
        "product_name": "samsung watch - T5",
        "amount": 52000,
    },
    {
        "year": 2014,
        "company": "jio",
        "product_name": "watch from bolt",
        "amount": 52000,
    },
    {
        "year": 2014,
        "company": "jio",
        "product_name": "Type 2 diabetes",
        "amount": 52000,
    },
    {
        "year": 2014,
        "company": "jio",
        "product_name": "(131)I-MIP 1095",
        "amount": 52000,
    },
]

# Insert the data into Weaviate
for record in sales_data:
    client.data_object.create(data_object=record, class_name="Sale")

In [ ]:
query_result = client.query.get(
    class_name="Sale",
    properties=["product_name", "year", "amount", "company"]
).with_near_text(
{
    'concepts': ["list Jio I MIP 1095 products"],
    "certainty": 0.7  # Set a higher threshold for more precise matches
}

 ).do()

print(query_result)

# Display the results
for sale in query_result["data"]["Get"]["Sale"]:
    print(f"Product: {sale['product_name']}")
